adding the raw inference outputs to the cropped_mosaics

In [ ]:
import rasterio
import numpy as np
from skimage.transform import resize

with open('tile_names_inference.txt', 'r') as file:
    filenames = file.read().splitlines()

for filename in filenames:
    # Load the TIFF file
    with rasterio.open(f'../../data_preprocessing/cropped_mosaic/{filename}_cropped_mosaic.tif') as src:
        # Read the existing bands
        bands = src.read()
        print("cropped_mosaic shape")
        print(bands.shape)

        # Load the NumPy array
        array = np.load(f'inference_output/inference_{filename}.npy')
        # print("inference_ensemble shape")
        # print(array.shape)
        array = np.mean(array, axis=0, keepdims=True)
        data = array[0]
        # print("averaged_inference shape")
        # print(data.shape)
        data = np.moveaxis(data, 2, 0)

        resized_data = resize(data, bands.shape)
        print("resized_data shape")
        print(resized_data.shape)

        # Check if the dimensions of the array match the TIFF file
        if bands.shape[1:] != resized_data.shape[1:]:
            raise ValueError("Array dimensions do not match the TIFF file")

        difference = np.where(bands[0] == -9999, 0 - resized_data[0], bands[0] - resized_data[0])
        difference = np.expand_dims(difference, axis=0)
        # Create a new array with the additional band
        new_bands = np.concatenate((bands, resized_data, difference), axis=0)

        profile = src.profile
        profile['count'] = 5

        '''
        band 1: icesat agbd
        band 2: icesat std
        band 3: predicted agbd
        band 4: ensemble std
        band 5: difference between icesat and predicted agbd
        
        '''

        # Create a new TIFF file with the additional band
        with rasterio.open(f'inference_output/inference_{filename}.tif', 'w', **profile) as dst:
            dst.write(new_bands)
        print(f'======== {filename} done ========')
